In [1]:
import pandas as pd
import struct
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
#dos = pd.read_csv('/content/drive/MyDrive/ACT Detection System Team/datasets/DoS_dataset.csv')
fuzzy = pd.read_csv('/content/drive/MyDrive/ACT Detection System Team/datasets/Fuzzy_dataset.csv')
#gear = pd.read_csv('/content/drive/MyDrive/ACT Detection System Team/datasets/gear_dataset.csv')
#rpm = pd.read_csv('/content/drive/MyDrive/ACT Detection System Team/datasets/RPM_dataset.csv')

In [10]:
#Make a function to turn hex to binary columns
# need to do split somehow
def hexToBinary(item):
  #return np.base_repr(int(item, 16), base=2)
  return np.binary_repr(int(item, 16), width=16)

In [11]:
#Curr = fuzzy
Curr = rpm
#Curr = gear
#Curr = dos

NameError: name 'rpm' is not defined

In [ ]:
#Same things but for RPM DataSet
col_names = ['Timestamp', 'CAN_ID', 'DLC', 'Data0','Data1','Data2','Data3','Data4','Data5','Data6', 'Data7', 'Flag']
Curr.columns = col_names

CurrProcessed = Curr.copy()

idsCurr = CurrProcessed['CAN_ID']

idsCurr = idsCurr.apply(hexToBinary)

In [ ]:
newCols = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15']
cols = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16']

splitCurrIds = idsCurr.str.split(pat="", n=16, expand=True)

splitCurrIds.columns = cols

splitCurrIds = splitCurrIds.drop('0', axis=1)

splitCurrIds.columns = newCols

In [ ]:
CurrIds2 = splitCurrIds.astype(int)
CurrIds2 = np.float32(CurrIds2)

numSplitCurr = CurrIds2.shape[0] // 16

CurrIds2 = CurrIds2[0:(numSplitCurr * 16)]
preChunksCurr = np.asarray(CurrIds2)
framesCurr = np.array_split(preChunksCurr, numSplitCurr)

In [ ]:
flagsCurr = CurrProcessed['Flag']
preSplitCurr = np.asarray(flagsCurr)

In [ ]:
flagsCurr = CurrProcessed['Flag']
preSplitCurr = np.asarray(flagsCurr)
preSplitCurr = preSplitCurr[:numSplitCurr * 16]
labels1Curr = np.array_split(preSplitCurr, numSplitCurr)
#labels2Rpm = np.resize(flagsRpm.T, [numSplitRpm, 16])

In [ ]:
finalLabelsCurr = []
for i in range(len(labels1Curr)):
  if 'T' in labels1Curr[i]:
    finalLabelsCurr.append(1)
  else:
    finalLabelsCurr.append(0)

In [ ]:
list_of_tensorsCurr = [torch.tensor(frame) for frame in framesCurr]
framesCurr = torch.stack(list_of_tensorsCurr)
framesCurr = framesCurr[:,np.newaxis,:,:]

In [ ]:
list_of_labelsCurr = [torch.tensor(np.asarray(label)).to(device) for label in finalLabelsCurr]
labels_for_datasetCurr = torch.stack(list_of_labelsCurr)
frameDataCurr = torch.utils.data.TensorDataset(framesCurr, labels_for_datasetCurr)

In [ ]:
#Use Proper Naming Convention
#Fuzzy
#RPM
#Gear
#Dos
torch.save(frameDataCurr, '/content/drive/MyDrive/ACT Detection System Team/datasets/RPM_Tensor_Dataset.pt')